In [3]:
import numpy as np 
import pandas as pd 
from sklearn.model_selection import train_test_split
import os
from xgboost.sklearn import XGBClassifier
from sklearn.model_selection import RandomizedSearchCV
from sklearn.metrics import accuracy_score
from sklearn.metrics import f1_score

In [5]:
print(os.listdir('C:/Users/Abhishek/JUPYTER/Reorder prediction'))

['aisles.csv', 'departments.csv', 'instacart-market-basket-analysis.zip', 'New folder', 'orders.csv', 'order_products__prior.csv', 'order_products__train.csv', 'products.csv', 'sample_submission.csv']


In [6]:
df_orders                  = pd.read_csv('C:/Users/Abhishek/JUPYTER/Reorder prediction/orders.csv')
df_order_products_train    = pd.read_csv('C:/Users/Abhishek/JUPYTER/Reorder prediction/order_products__train.csv')
df_order_products_prior    = pd.read_csv('C:/Users/Abhishek/JUPYTER/Reorder prediction/order_products__prior.csv')
df_products                = pd.read_csv('C:/Users/Abhishek/JUPYTER/Reorder prediction/products.csv')
df_departments             = pd.read_csv('C:/Users/Abhishek/JUPYTER/Reorder prediction/departments.csv')
df_aisles                  = pd.read_csv('C:/Users/Abhishek/JUPYTER/Reorder prediction/aisles.csv')
df_sample_submission       = pd.read_csv('C:/Users/Abhishek/JUPYTER/Reorder prediction/sample_submission.csv')

In [7]:
df_orders_products = pd.merge(df_order_products_train , df_orders, on = 'order_id')

In [8]:
df_orders_products.dropna()

,order_id,product_id,add_to_cart_order,reordered,user_id,eval_set,order_number,order_dow,order_hour_of_day,days_since_prior_order
0,1,49302,1,1,112108,train,4,4,10,9.0
1,1,11109,2,1,112108,train,4,4,10,9.0
2,1,10246,3,0,112108,train,4,4,10,9.0
3,1,49683,4,0,112108,train,4,4,10,9.0
4,1,43633,5,1,112108,train,4,4,10,9.0
...,...,...,...,...,...,...,...,...,...,...
1384612,3421063,14233,3,1,169679,train,30,0,10,4.0
1384613,3421063,35548,4,1,169679,train,30,0,10,4.0
1384614,3421070,35951,1,1,139822,train,15,6,10,8.0
1384615,3421070,16953,2,1,139822,train,15,6,10,8.0


In [9]:
X = df_orders_products[['user_id','product_id','order_id',
                        'add_to_cart_order','order_number','order_dow','order_hour_of_day']]
y = df_orders_products.reordered

In [10]:
X_train, X_test, y_train, y_test = train_test_split(X,y,test_size=0.2, random_state=1)

In [11]:
xgb = XGBClassifier() 

param_grid = {
        
        'max_depth': [1,2,3,4],
        'learning_rate': [0.001, 0.01, 0.1, 0.2, 0,3],
        'subsample': [0.5, 0.6, 0.7, 0.8, 0.9, 1.0],
        'min_child_weight': [0.5, 1.0, 3.0, 5.0, 7.0, 10.0],
        'gamma': [0, 0.25, 0.5, 1.0],
        'reg_lambda': [0.1, 1.0, 5.0, 10.0, 50.0, 100.0],
        'n_estimators': [100]}
clf = RandomizedSearchCV(xgb, param_grid, n_iter=10,
                            n_jobs= -1, verbose=2, cv=3, random_state=1)

In [12]:
clf.fit(X_train, y_train) 

Fitting 3 folds for each of 10 candidates, totalling 30 fits


[Parallel(n_jobs=-1)]: Using backend LokyBackend with 4 concurrent workers.
[Parallel(n_jobs=-1)]: Done  30 out of  30 | elapsed: 12.0min finished


RandomizedSearchCV(cv=3, error_score='raise-deprecating',
                   estimator=XGBClassifier(base_score=0.5, booster='gbtree',
                                           colsample_bylevel=1,
                                           colsample_bynode=1,
                                           colsample_bytree=1, gamma=0,
                                           learning_rate=0.1, max_delta_step=0,
                                           max_depth=3, min_child_weight=1,
                                           missing=None, n_estimators=100,
                                           n_jobs=1, nthread=None,
                                           objective='binary:logistic',
                                           random_state=0, reg_alpha=0...
                   param_distributions={'gamma': [0, 0.25, 0.5, 1.0],
                                        'learning_rate': [0.001, 0.01, 0.1, 0.2,
                                                          0, 3],
      

In [13]:
y_pred = clf.predict(X_test)
acc = accuracy_score(y_test,y_pred) 

print(acc)

0.6549558723693143


In [14]:
f1_score(y_test,y_pred)

0.7388853211635946